In [1]:
import numpy as np
from numpy import pi, cos,sin,tan,sqrt
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons

%matplotlib widget

from ipywidgets import *

def fun_n(sigma_t, theta):
    sigma_n = 1/2 * (sigma_t[0,0] + sigma_t[1,1]) + \
              1/2 * (sigma_t[0,0] - sigma_t[1,1]) * cos(2 * theta) + \
              sigma_t[0,1] * sin(2 * theta)
    tau_n   = -1/2 * (sigma_t[0,0] - sigma_t[1,1]) * sin(2 * theta) + \
              sigma_t[0,1] * cos(2 * theta)
    return [sigma_n,tau_n]
    
def fun_sigma_avg(sigma_t):
    sigma_avg = 1/2*(sigma_t[0,0] + sigma_t[1,1])
    return [sigma_avg,0]

def fun_radius(sigma_t):
    radius = sqrt((1/2*(sigma_t[0,0] - sigma_t[1,1]))**2+sigma_t[0,1]**2)
    return radius

def fun_principal_stress(sigma_avg, radius):
    return [[sigma_avg[0]+radius,0],[sigma_avg[0]-radius,0]]

In [5]:
def calc(sigma_x=10.0,sigma_y=20.0,tau_xy=10.0):
    sigma_t = np.array([[sigma_x,tau_xy],[tau_xy, sigma_y]])
    sigma_avg        = fun_sigma_avg(sigma_t)
    radius           = fun_radius(sigma_t)
    principal_stress = fun_principal_stress(sigma_avg,radius)

    print ('sigma: ')
    print (sigma_t)
    print ('sigma average: ')
    print (sigma_avg)
    print ('radius: ')
    print (radius)
    print ('principal stress: ')
    print (principal_stress)

    theta=np.linspace(0,pi,60)
    [sigma_n, tau_n] = fun_n(sigma_t, theta)
    line.set_xdata(sigma_n)
    line.set_ydata(tau_n)
    
    s1.set_xdata([sigma_t[1,1], sigma_t[0,0]])
    s1.set_ydata([sigma_t[0,1], -sigma_t[0,1]])
    
    #sc.set_offsets(sigma_avg)
    fig.canvas.draw_idle()
    
    
sigma_x_s = FloatSlider(
                value=10.00,
                min=-25.00,
                max=25.00,
                step=1.0,
                description=r'$\sigma_\mathrm{x}$',
                disabled=False,
                continuous_update=True
            )
sigma_y_s = FloatSlider(
                value=20.00,
                min=-25.00,
                max=25.00,
                step=1.0,
                description=r'$\sigma_\mathrm{y}$',
                disabled=False,
                continuous_update=True
            )
tau_xy_s = FloatSlider(
                value=10.00,
                min=-25.00,
                max=25.00,
                step=1.0,
                description=r'$\tau_\mathrm{xy}$',
                disabled=False,
                continuous_update=True
            )

# create a slider
interact(calc, sigma_x = sigma_x_s, sigma_y = sigma_y_s, 
         tau_xy  = tau_xy_s)


sigma_x=10.0
sigma_y=20.0
tau_xy=10.0

sigma_t = np.array([[sigma_x,tau_xy],[tau_xy, sigma_y]])
theta=np.linspace(0,pi,60)
[sigma_n, tau_n] = fun_n(sigma_t, theta)

fig, ax = plt.subplots()  # Create a figure and an axes.
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.resizable = False


line, = ax.plot(sigma_n,tau_n)  # Plot some data on the axes.

l = 30
ax.set_aspect(1)
ax.set_xlim([-l,l])
ax.set_ylim([-l,l])

s1, = ax.plot([sigma_t[1,1], sigma_t[0,0]], [sigma_t[0,1], -sigma_t[0,1]], c='grey')
sc = ax.scatter(fun_sigma_avg(sigma_t), c='blue')

ax.plot([-l,l], [0,0],  lw=0.5, c='black')
ax.plot([0,0],  [-l,l], lw=0.5, c='black')

# Move left y-axis and bottim x-axis to centre, passing through (0,0)
ax.spines['left'].set_position('center')
ax.spines['bottom'].set_position('center')

# Eliminate upper and right axes
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')

# Show ticks in the left and lower axes only
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')

ax.set_xlabel(r'$\tau_\mathrm{n}$')
ax.set_ylabel(r'$\sigma_\mathrm{n}$')

ax.xaxis.set_label_position("top")
ax.yaxis.set_label_position("right")

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …